<table style="font-size: 1em; padding: 0; margin: 0;">

<tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #182AEB; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #ffffff;">Deep Learning </span> for Satellite Image Classification</span> (Manning Publications)<br/>by <em>Daniel Buscombe</em></strong><br/><br/>
        <strong>> Chapter 4: Deliverable Solution </strong><br/>
    </p>   

#### Preliminaries for Colab

Like Part 3, below are some convenience functions for those working on Google Colab with a GPU runtime

In [2]:
colab = 0
#colab = 1

if colab==1:
    %tensorflow_version 2.x
    #!pip install --default-timeout=1000 tensorflow-gpu==2.0   

     |████████████████████████████████| 380.8MB 36kB/s 
     |████████████████████████████████| 3.8MB 6.3MB/s 
     |████████████████████████████████| 450kB 36.6MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existin

In [0]:
#%tensorflow_version 2.x

After restarting your runtime, make sure you have access to a GPU with Tensorflow 2

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
False


Convenience functions if you need to download example (minimal) imagery sets derived from NWPU and Sentinel-2 cloudless:


In [0]:
# from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


#s2 cloudless imagery
file_id = '1iMfIjr_ul49Ghs2ewazjCt8HMPfhY47h'
destination = 's2cloudless_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#s2 cloudless labels
file_id = '1c7MpwKVejoUuW9F2UaF_vps8Vq2RZRfR'
destination = 's2cloudless_label_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)


#nwpu imagery
file_id = '1gtuqy1VlU8-M5IEMnmiSuTlI5PxQPnGB'
destination = 'nwpu_images.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#nwpu labels
file_id = '1W5LGbcYAcFbG5YjLgX_ekBn0u5Rno35x'
destination = 'nwpu_label_images.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)                        

In [0]:
import zipfile
def unzip(f):
    """
    f = file to be unzipped
    """    
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()
        
if colab==1:
    unzip('s2cloudless_imagery.zip')
    unzip('s2cloudless_label_imagery.zip')   
    unzip('nwpu_images.zip')
    unzip('nwpu_label_images.zip')       

#### Setting up the model

We'll pick up from the end of the Part 3 deliverable, where you should have test and train generator functions and hyperparameters set.

You should have augmented images and associated labels for both sentinel2-cloudless and NWPU data

Define the IOU function and Unet model:

In [0]:
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
import numpy as np
import json, os
from random import shuffle
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

In [0]:
def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = tf.keras.backend.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

def unet(sz = (512, 512, 3)):
    inputs = Input(sz)
    _ = inputs
  
    #down sampling 
    f = 8
    layers = []
  
    for i in range(0, 6):
      _ = Conv2D(f, 3, activation='relu', padding='same') (_)
      _ = Conv2D(f, 3, activation='relu', padding='same') (_)
      layers.append(_)
      _ = MaxPooling2D() (_)
      f = f*2
    ff2 = 64 
  
    #bottleneck 
    j = len(layers) - 1
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_)
    _ = Concatenate(axis=3)([_, layers[j]])
    j = j -1 
  
    #upsampling 
    for i in range(0, 5):
      ff2 = ff2//2
      f = f // 2 
      _ = Conv2D(f, 3, activation='relu', padding='same') (_)
      _ = Conv2D(f, 3, activation='relu', padding='same') (_)
      _ = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_)
      _ = Concatenate(axis=3)([_, layers[j]])
      j = j -1 
    
    #classification 
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    outputs = Conv2D(1, 1, activation='sigmoid') (_)
  
    #model creation 
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [mean_iou])
  
    return model  


model = unet()

Create the class for training process plot, like shown in Part 4. One for each dataset. First, the augmented NWPU imagery

In [0]:
class PlotLearningNWPU(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.logs = []
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('mean_iou'))
        self.val_acc.append(logs.get('val_mean_iou'))
        self.i += 1
        print('i=',self.i,'loss=',logs.get('loss'),'val_loss=',logs.get('val_loss'),'mean_iou=',logs.get('mean_iou'),'val_mean_iou=',logs.get('val_mean_iou'))
        
        #choose a random test image and preprocess
        path = np.random.choice(test_filesNWPU)
        infile = f'nwpu_images/data/{path}'
        raw = Image.open(infile)
        raw = np.array(raw.resize((512, 512)))/255.
        raw = raw[:,:,0:3]
        
        #predict the mask 
        pred = 255*model.predict(np.expand_dims(raw, 0)).squeeze()
        print(np.max(pred))
                
        #mask post-processing 
        msk  = (pred>60).astype('int') #100       
        msk = np.stack((msk,)*3, axis=-1)
        
        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()

def build_callbacksNWPU():
        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='nwpu_unet.h5', verbose=0, save_best_only=True, save_weights_only=True)
        callbacks = [checkpointer, PlotLearningNWPU()]
        return callbacks

and a separate one for the S2 imagery

In [0]:
class PlotLearningS2(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.logs = []
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('mean_iou'))
        self.val_acc.append(logs.get('val_mean_iou'))
        self.i += 1
        print('i=',self.i,'loss=',logs.get('loss'),'val_loss=',logs.get('val_loss'),'mean_iou=',logs.get('mean_iou'),'val_mean_iou=',logs.get('val_mean_iou'))
        
        #choose a random test image and preprocess
        path = np.random.choice(test_filesS2)
        infile = f's2cloudless_imagery/data/{path}'
        raw = Image.open(infile)
        raw = np.array(raw.resize((512, 512)))/255.
        raw = raw[:,:,0:3]
        
        #predict the mask 
        pred = 255*model.predict(np.expand_dims(raw, 0)).squeeze()
        print(np.max(pred))
                
        #mask post-processing 
        msk  = (pred>60).astype('int') #100       
        msk = np.stack((msk,)*3, axis=-1)
        
        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()

def build_callbacksS2():
        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='s2cloudless_unet.h5', verbose=0, save_best_only=True, save_weights_only=True)
        callbacks = [checkpointer, PlotLearningS2()]
        return callbacks

Define image generator files for each data set

In [0]:
def image_batch_generatorS2(files, batch_size = 32, sz = (512, 512)):
  
  while True: # this is here because it will be called repeatedly by the training function
    
    #extract a random subset of files of length "batch_size"
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs (x) and outputs (y)
    batch_x = []
    batch_y = []
    
    #cycle through each image in the batch
    for f in batch:

        #preprocess the raw images 
        rawfile = f's2cloudless_imagery/data/{f}'
        raw = Image.open(rawfile)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
            raw = np.stack((raw,)*3, axis=-1)

        else:
            raw = raw[:,:,0:3]
            
        #get the image dimensions, find the min dimension, then square the image off    
        nx, ny, nz = np.shape(raw)
        n = np.minimum(nx,ny)
        raw = raw[:n,:n,:] 
            
        batch_x.append(raw)
        
        #get the masks. 
        maskfile = rawfile.replace('s2cloudless_imagery','s2cloudless_label_imagery')+'_mask.jpg'
        mask = Image.open(maskfile)
        # the mask is 3-dimensional so get the max in each channel to flatten to 2D
        mask = np.max(np.array(mask.resize(sz)),axis=2)
        # water pixels are always greater than 100
        mask = (mask>100).astype('int')
        
        mask = mask[:n,:n]

        batch_y.append(mask)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255. #divide image by 255 to normalize
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3) #add singleton dimension to batch_y

    yield (batch_x, batch_y) #yield both the image and the label together

In [0]:
def image_batch_generatorNWPU(files, batch_size = 32, sz = (512, 512)):
  
  while True: # this is here because it will be called repeatedly by the training function
    
    #extract a random subset of files of length "batch_size"
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs (x) and outputs (y)
    batch_x = []
    batch_y = []
    
    #cycle through each image in the batch
    for f in batch:

        #preprocess the raw images 
        rawfile = f'nwpu_images/data/{f}'
        raw = Image.open(rawfile)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
            raw = np.stack((raw,)*3, axis=-1)

        else:
            raw = raw[:,:,0:3]
            
        #get the image dimensions, find the min dimension, then square the image off    
        nx, ny, nz = np.shape(raw)
        n = np.minimum(nx,ny)
        raw = raw[:n,:n,:] 
            
        batch_x.append(raw)
        
        #get the masks. 
        maskfile = rawfile.replace('nwpu_images','nwpu_label_images')+'_mask.jpg'
        mask = Image.open(maskfile)
        # the mask is 3-dimensional so get the max in each channel to flatten to 2D
        mask = np.max(np.array(mask.resize(sz)),axis=2)
        # water pixels are always greater than 100
        mask = (mask>200).astype('int')
        
        mask = mask[:n,:n]

        batch_y.append(mask)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255. #divide image by 255 to normalize
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3) #add singleton dimension to batch_y

    yield (batch_x, batch_y) #yield both the image and the label together

Create two models, one for each data set

In [0]:
modelNWPU = unet()
modelS2 = unet()

In [0]:
batch_size = 8

prop_train = 0.6

#### Sentinel2-cloudless ("S2") imagery

Define the test and train steps like in Part 4 and run the model training

In [14]:
all_files = os.listdir('s2cloudless_imagery/data')
shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_filesS2 = all_files[0:split]
test_filesS2  = all_files[split:]

train_stepsS2 = len(train_filesS2) //batch_size
test_stepsS2 = len(test_filesS2) //batch_size

print(train_stepsS2)
print(test_stepsS2)

train_generatorS2 = image_batch_generatorS2(train_filesS2, batch_size = batch_size)
test_generatorS2  = image_batch_generatorS2(test_filesS2, batch_size = batch_size)

3
2


In [15]:
historyS2 = model.fit_generator(train_generatorS2, 
                    epochs = 100, steps_per_epoch = train_stepsS2,
                    validation_data = test_generatorS2, validation_steps = test_stepsS2,
                    callbacks = build_callbacksS2(), verbose = 0,
                    use_multiprocessing=True)

Output hidden; open in https://colab.research.google.com to view.

#### NWPU imagery

Define the test and train steps like in Part 4 and run the model training

In [16]:
all_files = os.listdir('nwpu_images/data')
shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_filesNWPU = all_files[0:split]
test_filesNWPU  = all_files[split:]

train_stepsNWPU = len(train_filesNWPU) //batch_size
test_stepsNWPU = len(test_filesNWPU) //batch_size

print(train_stepsNWPU)
print(test_stepsNWPU)

train_generatorNWPU = image_batch_generatorNWPU(train_filesNWPU, batch_size = batch_size)
test_generatorNWPU  = image_batch_generatorNWPU(test_filesNWPU, batch_size = batch_size)

42
28


In [0]:
historyNWPU = model.fit_generator(train_generatorNWPU, 
                    epochs = 100, steps_per_epoch = train_stepsNWPU,
                    validation_data = test_generatorNWPU, validation_steps = test_stepsNWPU,
                    callbacks = build_callbacksNWPU(), verbose = 0,
                    use_multiprocessing=True)

In [0]:
# summarize history for iou
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.plot(historyS2.history['mean_iou'],'k',lw=1)
plt.plot(historyS2.history['val_mean_iou'],'r',lw=1)
plt.ylim(0,1)
plt.axhline(y=0.85)
plt.title('S2 model IoU')
plt.ylabel('IoU')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
plt.plot(historyNWPU.history['mean_iou'],'k',lw=1)
plt.plot(historyNWPU.history['val_mean_iou'],'r',lw=1)
plt.title('NWPU model IoU')
plt.ylim(0,1)
plt.axhline(y=0.85)
plt.ylabel('IoU')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

#### Test NWPU model on S2 imagery

First, set up a new generator function to generate batches of S2 augmented imagery and associated labels. Then, define a function that will use the model (trained on NWPU imagery) to estimate the binary semantic segmentation mask for 100 images. Finally, we'll look at the mean IOU score, and their distribution

In [0]:
# rerun the generator function to start with a fresh set from the beginning of the list
test_generatorS2  = image_batch_generatorS2(test_filesS2, batch_size = batch_size)

# we need to change the IOU function a little because the inputs are slightly different shapes than used in the training
def mean_iou(y_true, y_pred):
    yt0 = y_true.squeeze()
    yp0 = tf.keras.backend.cast(y_pred.squeeze() > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

# a function for getting an estimated water mask from an input image, and IOU score evaluated against the real mask
def get_pred(x, y):
    #predict the mask 
    pred = model.predict(np.expand_dims(x, 0))
    
    #mask post-processing 
    msk  = pred.squeeze()
    # binarize
    msk[msk >= 0.5] = 1 
    msk[msk < 0.5] = 0
    # return the prediction and the IOU score of the prediction
    return msk, mean_iou(y, msk)

Let's see how this will work:

In [0]:
#get a batch of S2 imagery and labels
x, y = next(test_generatorS2) 
#get the predicted mask and iou score for the first
ypred, iou = get_pred(x[0], y[0]) 

#make a plot side-by-side of label ...
plt.subplot(121)
plt.imshow(x[0])
plt.imshow(y[0].squeeze(), alpha=0.5, cmap='gray')
plt.axis('off')

# ... and estimated label
plt.subplot(122)
plt.imshow(x[0])
plt.imshow(ypred.squeeze(), alpha=0.5, cmap='gray')
plt.title(str(iou.numpy())[:4])
plt.axis('off')

In [0]:
IOU = [] #initialize list
counter = 0 #initialize counter
while counter < 100: # compare 100 images
    x, y = next(test_generatorS2)
    ypred, iou = get_pred(x[0], y[0])
    IOU.append(iou) #update list

    if counter % 5 == 0: #print every 5th comparison
        plt.figure(figsize=(4,6))
        plt.subplot(121)
        plt.imshow(x[0])
        plt.imshow(y[0].squeeze(), alpha=0.5, cmap='gray')
        plt.axis('off')

        plt.subplot(122)
        plt.imshow(x[0])
        plt.imshow(ypred.squeeze(), alpha=0.5, cmap='gray')
        plt.title(str(iou.numpy())[:4])
        plt.axis('off')
        print(counter)  
        
    counter += 1 #update counter

Print the mean IOU score to screen and make a boxplot of the distribution of IOU scores

In [0]:
print(np.mean(IOU))

In [0]:
%matplotlib inline
fliers = dict(markerfacecolor='g', marker='p')
plt.figure(figsize=(8,4))
plt.boxplot(IOU, flierprops=fliers, vert=False, whis=0.75)
plt.xlabel('IOU scores')

This baseline comparison isn't brilliant but it is pretty good considering it hasn't yet been optimized AND it was built on a completely different set of images (NWPU). In deep learning, this concept of training a model on one set of imagery and applying to another, is called `transfer learning` and is very common in modern machine learning workflows.

We'll see in the final part many strategies for improving this accuracy. This distribution of IOU scores is the baseline against which we will improve upon